## Video Preview Creator (adapted for VIS 2021)

Creates a custom title slide for each author submitted video and puts it in front of the submitted video (approx. 5 sec title + 25 sec video)

### Requirements

system installs:


cairo : check https://cairocffi.readthedocs.io/en/stable/overview.html#installing-cairocffi

ffmpeg: should be installed somewhere in the system path (such that running `ffmpeg` from the command line works.)

*make sure they are in the system path


And all the following python packages

In [300]:
import pandas as pd

import os
import sys
import click
import cairocffi as cairo
import subprocess

import subprocess
import yaml
import wave
import os
import re

In [301]:
# [2022] convert mp4 file name to srt file name for srt subtitles, remove this code if srt file path is provided in the meta file
def replaceSRT(s):
    if s.endswith('mp4'):
        i = s.index('mp4')
        SRTFileName = s[0:i] + 'srt'
        return SRTFileName
    return s

#### Redapted from the previous year script

In [302]:
def probe( filename ):
    '''get some information about a video file'''

    proc = subprocess.Popen( [
            'ffprobe', 
            '-show_format',
            '-show_streams',
            '-print_format', 'json',
            '-i', filename
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE )

    out, err = proc.communicate()

    if proc.returncode:
        return None

    raw_info = yaml.full_load(out)

    info          = dict()                
    info['video'] = dict()
    info['audio'] = dict()

    info['duration'] = float( raw_info['format']['duration'] )
    info['size']     = float( raw_info['format']['size'] ) / 1048576.

    for s in raw_info['streams']:

        if s['codec_type'] == 'video':
            #print(s)
            info['video']['codec']  = s['codec_name']
            info['video']['format'] = s['pix_fmt']
            info['video']['width']  = int( s['width'] )
            info['video']['height'] = int( s['height'] )
            #info['video']['aspect'] = s['display_aspect_ratio']
            info['video']['fps']    = s['r_frame_rate']
            
        elif s['codec_type'] == 'audio':
            
            info['audio']['codec'] = s['codec_name']

    if 'codec' not in info['audio']:
        info['audio']['codec'] = '-'

    return info


In [303]:
def generate_vp(title_data):
   
    final = cairo.ImageSurface(cairo.FORMAT_ARGB32, 1920, 1080)
    ctx = cairo.Context(final)

    def linebreak(text, max_width, sep=' '):
        lines = []

        for text in text.split("\\n"):
            tokens = text.split(sep)
            line = []

            for token in tokens:
                extline = line + [token]
                width = ctx.text_extents(sep.join(extline))[2]

                if width > max_width:
                    lines.append((sep.join(line) + sep))
                    line = [token]
                else:
                    line = extline

            lines.append(sep.join(line))

        return [l.strip() for l in lines]

    def draw_text(text, x, y, size):

        ctx.set_font_size(size)

        y += ctx.font_extents()[0]

        for num, line in enumerate(text):
            ctx.move_to(x, y)
            ctx.show_text(line)
            y = y + 1.2 * size

    img_bkgnd = cairo.ImageSurface.create_from_png( 'preview-background 2022.png' )

    # clear image
    ctx.rectangle( 0, 0, 1920, 1080 )
    ctx.set_source_rgba( 1, 1, 1, 1 )
    ctx.fill()

    ctx.set_source_surface( img_bkgnd )
    ctx.paint()

    # draw paper type
    ctx.select_font_face( "Tahoma Bold" )
    ctx.set_font_size( 30 )
    ctx.set_source_rgba( 0, 0, 0, 1 )

    draw_text( [title_data[0].upper()], 300, 500, 50 )

    # draw title
    # [2022] update typography
    ctx.select_font_face( "Tahoma" )
    ctx.set_source_rgba( 0.64, 0.04, 0.21, 1 )

    for fontsize in range( 60, 50, -2 ):

        ctx.set_font_size( fontsize )
        lines = linebreak( title_data[1], 1500 )

        if len(lines) < 4:
            break

    if len(lines) > 2:
        ctx.translate( 0,  0.3*fontsize )
    if len(lines) < 1:
        ctx.translate( 0,  0.3*fontsize )

    # [2022] update typography position
    draw_text( lines, 300, 750-1.2*fontsize*len(lines), fontsize )

    # draw authors
    # [2022] update typography
    ctx.select_font_face( "Tahoma" )
    ctx.set_source_rgba( 0.114, 0.192, 0.376, 1 )

    for author_fontsize in range( 50, 40, -2 ):

        ctx.set_font_size( author_fontsize )
        lines = linebreak( title_data[2], 1500, ',' )

        if len(lines) < 4:
            break

    # [2022] update typography position        
    draw_text( lines, 300, 760, 35 )
    
    # draw awards information
    # [2022] update typography
    if(title_data[6] == 'Best Paper' or title_data[6] == 'Honorable Mention'):
        ctx.select_font_face( "Tahoma" )
        ctx.set_source_rgba( 0.83, 0.686, 0.216, 1 )

        for session_fontsize in range( 60, 50, -2 ):

            ctx.set_font_size( session_fontsize )
            lines = linebreak( '*'+title_data[6], 1800, ',' )

            if len(lines) < 4:
                break

        # [2022] update typography position
        draw_text( lines, 300, 850, 40 )
    
    # draw session info
    # [2022] update typography
    ctx.select_font_face( "Tahoma" )
    ctx.set_source_rgba( 0, 0, 0, 1 )

    for session_fontsize in range( 80, 40, -2 ):

        ctx.set_font_size( session_fontsize )
        lines = linebreak( title_data[4], 1800, ',' )

        if len(lines) < 4:
            break
    
    # [2022] update typography position        
    draw_text( lines, 300, 850, 40 )

    png_file = 'tmp_yellow.png'

    final.write_to_png( png_file )
    
    # use FFMPEG to prepend image to video
    # filter syntax: https://ffmpeg.org/ffmpeg-filters.html#Filtergraph-syntax-1
    
    # check if preview already exists
    outfile = os.path.join( title_data[5] )

#     if os.path.exists( outfile ):
#         print( '  preview found in', outfile )
#         return outfile

    #input_file = find_video_file( meta['id'] )
    input_file = title_data[3]
    info = probe( input_file )
    
    # [2022] load SRT file as substitles
    subtitlePath = replaceSRT(title_data[3])

    filter  = '[0:v]trim=duration=5,fade=t=out:st=4.5:d=0.5[v0];'
    filter += 'aevalsrc=0:d=5[a0];'
    # [2022] add srt file into filter
    filter += '[1:v]subtitles=' + subtitlePath + ','
    filter += 'scale=w=1920:h=1080[v1];'


    if info['audio']['codec'] == '-':
        filter += 'aevalsrc=0:d=25[a1];'
    else:
        filter += '[1:a]anull[a1];'

    filter += '[v0][a0][v1][a1]concat=n=2:v=1:a=1[v][a]'

    cmd = [
        'ffmpeg',
        '-v', 'error',
        '-framerate', info['video']['fps'],
        '-f', 'image2pipe',
        '-vcodec', 'png',
        '-loop', '1',
        '-i', png_file,
        '-i', input_file,
        '-i', subtitlePath,
        
        '-filter_complex', filter,
        '-map', '[v]',
        '-map', '[a]',
        '-c:v', 'libx264',
        '-pix_fmt', 'yuv420p',
        '-profile:v', 'main',
        '-preset', 'slow',
        '-y', outfile
    ]

    try:
        p = subprocess.Popen( cmd )
        p.wait()
    except:
        # on error, delete the output file
        os.unlink( outfile )
        print( '  abort, deleted', outfile )
        raise


In [304]:
# [2022] update metadata file path
df = pd.read_excel(open('VP_2022_metadata-TEST.xlsx', 'rb'),sheet_name='VP_ShortPaper', engine='openpyxl')

In [305]:
df

,Submission ID,Type,Title,Authors,Contact Email,Award,Filename,Session Name,Session Time
0,1142,VIS Full Paper,MEDLEY: Intent-based Recommendations to Suppor...,"Aditeya Pandey, Arjun Srinivasan, Vidya Setlur",arjunsrinivasan@tableau.com,NaN,v-full-1142_Pandey_Preview.mp4,Session 1,0800-0810
1,1577,VIS Full Paper,FlowNL: Asking Flow Data in Natural Language,"ieying Huang, Yang Xi, Junnan Hu, Jun Tao",taoj23@mail.sysu.edu.cn,NaN,v-full_1577_Huang_Preview.mp4,Session 2,0900-0910


In [306]:
#remove entries with no files. probably authors did not submit
clean_df = df.loc[df['Filename'].notna()]

In [307]:
print('Available videos:', len(clean_df))
print('Total Submissions:', len(df))


Available videos: 2
Total Submissions: 2


In [308]:
input_vid_dir = 'video_dir/VIS Short/'
output_vid_dir = 'video_dir/VP_out_VIS Short/'

In [309]:
for index, row in clean_df.iterrows():

    video_metadata = [row['Type'],
                      row['Title'],
                      row['Authors'],
                      input_vid_dir + row['Filename'],
                      row['Session Name'] + ': ' + row['Session Time'],
                      #'Social Sciences, Software Tools, Journalism, and Storytelling: Friday, 0815 - 0830',
                      output_vid_dir + row['Filename'],
                      row['Award']
                     ]
    print(index, video_metadata, '\n\n')
    generate_vp(video_metadata)
    
    
    

0 ['VIS Full Paper', 'MEDLEY: Intent-based Recommendations to Support Dashboard Composition', 'Aditeya Pandey, Arjun Srinivasan, Vidya Setlur', 'video_dir/VIS Short/v-full-1142_Pandey_Preview.mp4', 'Session 1: 0800-0810', 'video_dir/VP_out_VIS Short/v-full-1142_Pandey_Preview.mp4', nan] 


subtitles=video_dir/VIS Short/v-full-1142_Pandey_Preview.srt
1 ['VIS Full Paper', 'FlowNL: Asking Flow Data in Natural Language', 'ieying Huang, Yang Xi, Junnan Hu, Jun Tao', 'video_dir/VIS Short/v-full_1577_Huang_Preview.mp4', 'Session 2: 0900-0910', 'video_dir/VP_out_VIS Short/v-full_1577_Huang_Preview.mp4', nan] 


subtitles=video_dir/VIS Short/v-full_1577_Huang_Preview.srt
